In [1]:
%matplotlib widget
# %matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook as tqdm
from matplotlib import rc
rc("font", family = "serif",size=20)
rc("figure",figsize=(9,6))
rc("figure",facecolor="white")
%config InlineBackend.figure_format = 'retina'

# %load_ext autoreload
# %autoreload 2
import skimage.io as io
import re
import os
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from skimage import io
from skimage.color import rgb2grey
from skimage.filters import sobel
from skimage import filters
def imread_convert(f):
    return rgb2grey(io.imread(f))
from skimage.segmentation import flood_fill
def reduce(arr,column,function=np.mean):
    #calc new dtype
    dtype = dict(arr.dtype.fields) #make a shallow copy
    dtype.pop(column)
    dtype.pop('name')
    
    #which identifying columsn will survive
    remaining_identifiers = [i for i in dtype.keys()  if i not in [column,'name','image']]
    
    #get number of unique entries in output
    unique = np.unique(arr[remaining_identifiers])
    # i think that this will preserve order??
    
    col = arr[column]
    #need unique remaining indentify columns
    
    
    out = np.zeros(len(unique),dtype= dtype)
    print(dtype)
    for i,v in enumerate(unique):
        ind = arr[remaining_identifiers]==v
        #get indices ind = col == i
        im = function(arr['image'][ind])

        out[i] = (*v,im)
    return out
def pos_timelapse(arr,pos=0):
    """
    Assumes the first column is 'pos' and the second column is time steps
    """
    idx = arr['pos']==pos
    fig = plt.figure(f"position {pos}")
    im = plt.imshow(arr[idx][0]['image'])
    def on_value_change(change):
        i = change['new']
        im.set_data(arr[idx][i]['image'])
        fig.canvas.draw()
    slider = widgets.IntSlider(min=0, max=np.sum(idx)-1, step=1, continuous_update=True)
    play = widgets.Play(min=0,max=np.sum(idx)-1, interval=200,_repeat=True,_playing=True,step=1)

    slider.observe(on_value_change, 'value')
    widgets.jslink((play, 'value'), (slider, 'value'))
    return widgets.HBox([play,slider])

def choose_pos_timelapse(arr,pos=0):
    """
    Assumes the first column is 'pos' and the second column is time steps
    """
    idx = arr['pos']==pos
    fig = plt.figure(f"position {pos}")
    im = plt.imshow(arr[idx][0]['image'])
    positions = np.unique(arr['pos'])
    pos_widget = widgets.Dropdown(
        options=positions,
        value=positions[0],
        description='Position:',
        disabled=False,
    )
    t_slider = widgets.IntSlider(min=0, max=np.sum(idx)-1, step=1)#, continuous_update=True)
    t_play = widgets.Play(min=0, max=np.sum(idx)-1,interval=200,_repeat=True,step=1)
    def redraw(*args):
        idx = arr['pos']==pos_widget.value
        i = t_slider.value
        im.set_data(arr[idx][i]['image'])
        fig.canvas.draw()
        
    def on_value_change(change):
        redraw()
#         i = change['new']
#         idx = arr['pos']==pos_widget.value
# #         print(i)
#         im.set_data(arr[idx][i]['image'])
#         fig.canvas.draw()
        
    def update_t_range(*args):
#         print(np.sum(arr['pos']==pos_widget.value))
        t_slider.max = np.sum(arr['pos']==pos_widget.value)-1    
        redraw()
    
    widgets.jslink((t_slider, 'value'),(t_play, 'value') )    
    pos_widget.observe(update_t_range,'value')
    t_slider.observe(on_value_change, 'value')
    time_box = widgets.HBox([t_play,t_slider])
    ui = widgets.VBox([pos_widget, time_box])
    return ui
def int_imshow_interact(arr,title_col=None):
#     play = widget.Play()
    i_widget = widgets.IntSlider(min=0,max=len(arr)-1,value=0)
    fig = plt.figure()
    ax = plt.gca()
    im = plt.imshow(arr[0]['image'])
    def update(i):
        if title_col is not None:
            ax.set_title(f"{title_col} = {arr[i][title_col]}")
        im.set_data(arr[i]['image'])
        fig.canvas.draw()
    interact(update,i=i_widget)
def milli2min(ms):
    return ms/(1000*60)
def milli2hr(ms):
    return ms/(1000*60*60)
def min2hr(min):
    return min/60
# int_imshow_interact(out,title_col='time')

# im = filters.gaussian(elevation_map,sigma=3)

tolerance = .01

# fig = plt.figure()
# ax = plt.gca()
class yikes:
    def __init__(self):
#         plt.imshow
        self.positions = []
        self.circles = []
        self.tolerance_widget = widgets.FloatText(
            value=tolerance,
            description='tolerance:',
            disabled=False,
            step=.1
        )
        recalc = widgets.Checkbox(
            value=False,
            description='Recalc on change',
            disabled=False
        )
        plt.clf()
        self.fig = plt.figure('yikes')
        self.ax = plt.gca()
        self.displayed = self.ax.imshow(im)
        
        self.tolerance_widget.observe(self.set_tol,'value')
        self.tolerance=.01
        self.ix = 0
        self.iy = 0

        self.fig.canvas.mpl_connect('button_press_event', lambda event: self.onclick(event)) # if you call self.onclick here the figure will render when object is created
    def onclick(self,event):
        print(self.tolerance)
        self.ix, self.iy = event.xdata, event.ydata
        self.filled = flood_fill(im,(int(self.iy),int(self.ix)),new_value=255,tolerance=self.tolerance)
        self.displayed.set_data(self.filled)

        c = plt.Circle((self.ix,self.iy),5,color='r')
        self.circles.append(c)
        self.ax.add_artist(c)
        self.positions.append((self.ix, self.iy))
        self.fig.canvas.draw()

    def set_tol(self,change):
        self.tolerance
        self.tolerance = change['new']
    def _ipython_display_(self):
        display(widgets.VBox([self.fig.canvas,self.tolerance_widget]))
import glob
def get_file_names(direc,pos="*"):
    """
    dir should be somethign like 'sunday1' or 'sunday1/' the / will be stripped.
    """
    return glob.glob(f"{direc.strip('/')}/images/{pos}*")

def process_file(f,prefix_len):
    cur_f = f[prefix_len:]
    idx1 = cur_f.find('_')
    int(i[:1])
def load_images(dir,files):
    prefix = f"{dir.strip('/')}/images/"
    offset = len(prefix)
    files_for_reading = [f[offset:] for f in files]
    print(files_for_reading)
    
    
    a = np.sort(np.array([(int(i[:1]),int(i[2:10]),float(i[13:-4]),f"{dir.strip('/')}images/"+i,imread_convert(f"{dir.strip('/')}/images/{i}")) for i in files_for_reading ],dtype=[("pos",np.int),("time",np.int),("z",np.float),("name","U40"),("image","(450,540)float")]))
    return a
def load_pos(dir,pos):
    files = get_files(dir,f"{pos}")
    return load_images(dir,files)

def process_file(f,prefix_len):
    
    cur_f = f[prefix_len:]
    idx1 = cur_f.index('_')
    idx2 = cur_f[idx1+1:].index('_')
    pos = cur_f[:idx1] # extract pos
    t = cur_f[idx1+1:idx2+idx1+1] # extract timestamp
    z = cur_f[idx2+idx1+4:-4] # extract z
    return int(pos),int(t),float(z),f,imread_convert(f)
def load_pos(files,prefix_len):
    return np.sort(np.array([process_file(f,prefix_len) for f in files]    ,dtype=[("pos",np.int),("time",np.int),("z",np.float),("name","U40"),("image","(450,540)float")]))
# plt.close('all')
class choose_best:
    def __init__(self,arr):
        self.arr = arr
        self.times = np.sort(np.unique(self.arr['time']))
#         print(self.times)
        self.cur_t = 0
        self.t_idx = self.arr['time']==self.times[self.cur_t]
        self.N_z_cur = np.sum(self.t_idx)-1
        self.z_slider = widgets.IntSlider(
            value=0,
            max = self.N_z_cur,
            description='z:',
            disabled=False,
            step=1,
        )
#         self.t_slider
        self.choose_button = widgets.Button(
            description='Choose as good z',
            disabled=False
        )
        self.next_button = widgets.Button(icon='step-forward')
        self.prev_button = widgets.Button(icon='step-backward')
        self.next_button.on_click(self.next_t)
        self.prev_button.on_click(self.prev_t)
#         plt.clf()
        self.fig = plt.figure('yikes')
        self.ax = plt.gca()
        self.displayed = self.ax.imshow(self.arr['image'][self.t_idx][0])
        
        self.z_slider.observe(self.redraw,'value')

        self.i = 0
        self.name_list = [None]*len(self.times)
        self.choose_button.on_click(self.chosen)
    def chosen(self,*args):
        self.name_list[self.cur_t]= self.arr['name'][self.t_idx][self.i]
        self.next_t()
    def prev_t(self,*args):
        self.cur_t-=1
        if self.next_button.disabled:
            self.next_button.disabled = False
        self.update_t()
    def next_t(self,*args):
        try:
            self.cur_t+=1
            self.update_t()
        except IndexError:
            self.next_button.disabled=True
            print("end of time steps")
#             pass
        
    def update_t(self):
        self.t_idx = self.arr['time']==self.times[self.cur_t]
        self.update_z_range()
        self.redraw()
    def update_z_range(self,*args):
        new_max = np.sum(self.t_idx)-1
        self.z_slider.max = new_max

    def redraw(self,*args):
        self.i = self.z_slider.value
        self.displayed.set_data(self.arr[self.t_idx][self.i]['image'])
        self.fig.canvas.draw()
    def get_filtered_list(self):
        """
        remove any Nones from the names list.
        this allows for the filtering out of time steps that are messed up for whatever reason
        """
        return [n for n in self.name_list if n is not None]
    def write_names_to_file(self):
        names = self.get_filtered_list()
        direc = names[0][:names[0].index('/')]
        fname = f"{direc}_names.txt"
        print(f'writing to {fname}')
        with open(fname,'a') as f:
            f.write('\n'.join(names))
            f.write('\n')
    def _ipython_display_(self):
        time_buttons = widgets.HBox([self.prev_button,self.next_button])
        display(widgets.VBox([self.fig.canvas,self.z_slider,self.choose_button,time_buttons]))


In [2]:
def gen_obj_for_time(direc,pos=0):
    f = get_file_names(direc,pos)
    print(f[0])
    prefix_len = len(f"{direc.strip('/')}/images/")
#    prefix_len = len(prefix)
    ims = load_pos(f,prefix_len)
    
#     t0 = p0['time']==times[0]
#     cb = ?
    return choose_best(ims)
plt.close('all')

In [11]:
cb = gen_obj_for_time('sunday1/',2)
cb

sunday1/images/2_165481093_z_1.75.jpg


In [12]:
cb.write_names_to_file()

writing to sunday1_names.txt


In [13]:
plt.close('all')